# Reading list:

[Building custom question-answering app using LangChain and Pinecone vector database](https://medium.com/ai-science/building-custom-question-answering-app-using-langchain-and-pinecone-vector-database-35f7855a5323)

In [1]:
# Install package:
# pip install "unstructured[pdf]" #### pip install "unstructured[all-docs]"

In [2]:
import os
# import openai
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.llms import OpenAI
# from langchain.chains.question_answering import load_qa_chain
from llama_index import SimpleDirectoryReader

from IPython.display import Markdown, display

import environ

# Read OpenAI key
env = environ.Env()
environ.Env.read_env()
API_KEY = env("OPENAI_API_KEY")

from utils import CHUNK_SIZE, CHUNK_OVERLAP

# Set paths
docs_path = "documents_pdf"
embedding_path = "vector_db"

/home/daniele/Desktop/Projects/chatbot_afa/.venv_chatbot_afa/lib/python3.10/site-packages/environ/environ.py:639: UserWarning: /tmp/ipykernel_7177/.env doesn't exist - if you're not configuring your environment separately, create one.
  warnings.warn(


## Llama  Index

In [3]:
docs_llama = SimpleDirectoryReader(input_dir=docs_path).load_data()

In [4]:
len(docs_llama)

774

In [5]:
docs_llama[0]

Document(id_='dd87666b-c847-430e-a9b2-8b7bceefc0fd', embedding=None, metadata={'page_label': '1', 'file_name': 'ba146349.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='592bb4a567ff530dbc0607be761d79990988384fdd21bdcda882ef64fc386f94', text=' Familienkasse\nWir helfen Familien.\nMerkblatt\nKinderzuschlag\nDer Zuschlag zum Kindergeld für Familien  \nmit kleinem Einkommen', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [6]:
docs_llama[0].metadata

{'page_label': '1', 'file_name': 'ba146349.pdf'}

In [7]:
docs_llama[0].get_text()

' Familienkasse\nWir helfen Familien.\nMerkblatt\nKinderzuschlag\nDer Zuschlag zum Kindergeld für Familien  \nmit kleinem Einkommen'

In [8]:
from llama_index.text_splitter import SentenceSplitter

text_splitter = SentenceSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)

In [9]:
text_chunks = []
text_metadata = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, page in enumerate(docs_llama):
    page_text = page.get_text()
    cur_text_chunks = text_splitter.split_text(page_text)
    text_chunks.extend(cur_text_chunks)
    text_metadata.append(page.metadata)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [ ]:
for i in range(10, 15):
    display(Markdown(text_chunks[i]))
    print(f"LEN: {len(text_chunks[20])}")
    print(text_metadata[i])
    print("="*50)

# Langchain

In [2]:
# load_docs functions to load documents using langchain function
def load_docs(path):
  """
  
  """
  loader = DirectoryLoader(path)
  documents = loader.load()
  return documents

documents = load_docs(docs_path)
len(documents)

The PDF <_io.BufferedReader name='documents_pdf/ba146349.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='documents_pdf/kg2-merkblattkindergeld_ba015394.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


19

In [87]:
len(documents[0].page_content)

60840

In [9]:
documents[0].metadata

{'source': 'documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf'}

In [156]:
# split the docs using recursive text splitter
def split_docs(documents, chunk_size=200, chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

# split the docs
docs = split_docs(documents, chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
print(len(docs))

946


In [157]:
print(docs[0].metadata)
print(docs[1].metadata)
print(docs[2].metadata)
print(docs[3].metadata)
print(docs[4].metadata)
print(docs[5].metadata)
print(docs[10].metadata)

{'source': 'documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf'}
{'source': 'documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf'}
{'source': 'documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf'}
{'source': 'documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf'}
{'source': 'documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf'}
{'source': 'documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf'}
{'source': 'documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf'}


In [146]:
docs[0]

Document(page_content='Merkblatt\n\nFörderung der beruflichen Weiterbildung für Arbeitnehmerinnen und Arbeitnehmer Dienste und Leistungen der Agentur für Arbeit\n\n6\n\nMusterstadthausen\n\nVorwort\n\nVorwort\n\nSie interessieren sich für eine berufliche Weiterbildung. Dieses Merkblatt informiert Sie über Fragen rund um die Förderung Ihrer beruflichen Weiterbildung durch die Agenturen für Arbeit /Jobcenter. Bitte lesen Sie das Merkblatt in Ihrem eigenen Interes- se zur Vermeidung von Nachteilen sorgfältig durch. Das Merkblatt kann leider nicht auf jede Einzelheit ein- gehen. Sollten Sie daher noch Fragen haben, auf die Sie hier keine Antwort finden, oder sollten Sie weitere Informati- onen wünschen, erteilen Ihnen die Mitarbeiterinnen und Mitarbeiter der für Ihren Wohnort zuständigen Agentur für Arbeit /Jobcenter gerne nähere Auskunft.\n\nSelbstverständlich können Sie in Ihrer Agentur für Ar- beit / Ihrem Jobcenter auch die für die Entscheidung über Ihren Leistungsanspruch maßgebenden 

In [177]:
for i in range(10, 15):
    display(Markdown(docs[i].page_content))
    print(f"Metadata: {docs[i].metadata['source']}")
    print(f"LEN: {len(docs[i].page_content)}")

    print("="*50)

1.1 Notwendigkeit der Weiterbildung

Eine berufliche Weiterbildung soll Ihre Vermittlungs- chancen deutlich verbessern. Bei der Förderung beruf- licher Weiterbildung werden Ihre Fähigkeiten, Ihr bis­ heriger beruflicher Werdegang und Vorkenntnisse und Ihre persönlichen Voraussetzungen wie Eignung und Mobilität berücksichtigt. Nach diesen Faktoren ent- scheidet die Agentur für Arbeit / das Jobcenter nach ausführlicher Be ratung, ob Sie zur beruflichen Einglie- derung eine Qualifizierung benötigen. Hierbei kommt der Aufnahmefähigkeit des Arbeitsmarktes und Ihrer Mobilitätsbereitschaft eine hohe Bedeutung zu. Die Notwendigkeit der Weiterbildung kann zur Beendigung oder Vermeidung der Arbeitslosigkeit anerkannt wer- den. Ziel ist Ihre dauerhafte Eingliederung in den ers- ten Arbeits markt. Wenn Sie nicht über einen Berufsabschluss verfügen, besteht unter bestimmten Voraussetzungen ein Rechtsanspruch auf Förderung des nachträglichen Er- werbs eines Berufsabschlusses.

1.2 Beratung

Zur Feststellung, ob Sie eine berufliche Weiterbildung benötigen, ist es unbedingt erforderlich, dass möglichst frühzeitig eine Beratung durch die Agentur für Arbeit / das Jobcenter erfolgt. Vereinbaren Sie dazu einen Termin mit Ihrer Agentur für Arbeit / Ihrem Jobcenter. Im Rahmen dieser Beratung werden Ihre Fragen im

11

1 Voraussetzungen zur Förderung

Zusammenhang mit der beruflichen Weiterbildung besprochen und das Vorliegen der Voraussetzungen für eine Förderung geklärt. Eventuell kann für die Fest- stellung ihrer Eignung auch ein Termin beim Ärztlichen Dienst oder Berufspsychologischen Service der Agen- tur für Arbeit sinnvoll sein.

1.3 Bildungsgutschein

Metadata: documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf
LEN: 1663


1.2 Beratung

Zur Feststellung, ob Sie eine berufliche Weiterbildung benötigen, ist es unbedingt erforderlich, dass möglichst frühzeitig eine Beratung durch die Agentur für Arbeit / das Jobcenter erfolgt. Vereinbaren Sie dazu einen Termin mit Ihrer Agentur für Arbeit / Ihrem Jobcenter. Im Rahmen dieser Beratung werden Ihre Fragen im

11

1 Voraussetzungen zur Förderung

Zusammenhang mit der beruflichen Weiterbildung besprochen und das Vorliegen der Voraussetzungen für eine Förderung geklärt. Eventuell kann für die Fest- stellung ihrer Eignung auch ein Termin beim Ärztlichen Dienst oder Berufspsychologischen Service der Agen- tur für Arbeit sinnvoll sein.

1.3 Bildungsgutschein

Liegen die Voraussetzungen für eine Förderung vor, erhalten Sie einen Bildungsgutschein, mit dem Ihnen die Übernahme der Weiterbildungskosten und ggf. die Weiterzahlung des Arbeitslosengeldes zugesichert wird. Der Bildungsgutschein gilt zeitlich befristet. Er kann auf eine bestimmte Region beschränkt werden. Im Bil- dungsgutschein ist immer ein bestimmtes Bildungsziel angegeben. Während der Gültigkeitsdauer des Bildungsgutscheines können Sie eine dem Bildungsgutschein entsprechende zugelassene Maßnahme auswählen. Die Maßnahmen finden Sie auf der Seite » www.arbeitsagentur.de/ kursnet über die KURSNET Einstiegsseite. Die für den ausgewählten Träger bestimmte Ausferti- gung des Bildungsgutscheins, mit der der Träger Ihre Aufnahme in die Maßnahme bestätigt, muss innerhalb des Gültigkeitszeitraumes und vor dem Beginn Ihrer Teilnahme bei der Agentur für Arbeit / dem Jobcenter eingereicht werden.

TIPP

Damit Ihnen die zustehenden Leistungen zeitnah be- willigt werden können, sollten Sie die erforderlichen Unterlagen rechtzeitig vor Beginn der Teilnahme bei Ihrer Agentur für Arbeit / Ihrem Jobcenter einreichen.

12

1 Voraussetzungen zur Förderung

1.4 Weiterbildungen im Auftrag von Agenturen

für Arbeit und Jobcentern

Metadata: documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf
LEN: 1919


TIPP

Damit Ihnen die zustehenden Leistungen zeitnah be- willigt werden können, sollten Sie die erforderlichen Unterlagen rechtzeitig vor Beginn der Teilnahme bei Ihrer Agentur für Arbeit / Ihrem Jobcenter einreichen.

12

1 Voraussetzungen zur Förderung

1.4 Weiterbildungen im Auftrag von Agenturen

für Arbeit und Jobcentern

Neben der Förderung über den Bildungsgutschein gibt es auch die Möglichkeit der Teilnahme an einer Auf- tragsmaßnahme. Hier führt ein Bildungsträger im Auf- trag der Agentur für Arbeit oder des Jobcenters eine Weiterbildung durch. Anstelle des Bildungsgutscheins, bei dem Sie den Bildungsträger selbst aussuchen kön- nen, erhalten Sie ein Angebot zur Teilnahme an einer konkreten Weiterbildung. Für Agenturen für Arbeit und Jobcenter gelten dabei folgende Unterschiede:

Agenturen für Arbeit und Jobcenter können Träger mit der Durchführung folgender Weiterbildungen beauf­ tragen: • Maßnahmen zum Erwerb von Grundkompetenzen

(siehe auch 1.7),

Maßnahmen in Kombination „Erwerb von Grundkom- petenzen“ und Weiterbildungen, die zu einem Ab- schluss in einem Ausbildungsberuf (Umschulungen) führen oder

Maßnahmen, die eine Weiterbildung in einem Be- trieb, die zu einem Abschluss in einem Ausbildungs- beruf führt (betriebliche Umschulung), begleiten (um- schulungsbegleitende Hilfen, siehe auch 1.8).

Zusätzlich können Jobcenter auch Träger mit der Durchführung beruflicher Weiterbildungsmaßnahmen beauftragen, wenn eine dem Bildungsziel entsprechen- de Weiterbildung in Ihrer Region bislang nicht verfüg- bar ist. Dies gilt grundsätzlich für alle Arten der Weiter- bildung (z. B. Anpassungsqualifizierungen, Umschulungen).

13

1 Voraussetzungen zur Förderung

1.5 Zulassung des Trägers und der Maßnahme

Der Maßnahmeträger und die angestrebte Maßnahme müssen für die Weiterbildungsförderung von einer fachkundigen Stelle nach der Akkreditierungs- und Zu- lassungsverordnung Arbeitsförderung zugelassen sein.

HINWEIS

Metadata: documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf
LEN: 1949


(siehe auch 1.7),

Maßnahmen in Kombination „Erwerb von Grundkom- petenzen“ und Weiterbildungen, die zu einem Ab- schluss in einem Ausbildungsberuf (Umschulungen) führen oder

Maßnahmen, die eine Weiterbildung in einem Be- trieb, die zu einem Abschluss in einem Ausbildungs- beruf führt (betriebliche Umschulung), begleiten (um- schulungsbegleitende Hilfen, siehe auch 1.8).

Zusätzlich können Jobcenter auch Träger mit der Durchführung beruflicher Weiterbildungsmaßnahmen beauftragen, wenn eine dem Bildungsziel entsprechen- de Weiterbildung in Ihrer Region bislang nicht verfüg- bar ist. Dies gilt grundsätzlich für alle Arten der Weiter- bildung (z. B. Anpassungsqualifizierungen, Umschulungen).

13

1 Voraussetzungen zur Förderung

1.5 Zulassung des Trägers und der Maßnahme

Der Maßnahmeträger und die angestrebte Maßnahme müssen für die Weiterbildungsförderung von einer fachkundigen Stelle nach der Akkreditierungs- und Zu- lassungsverordnung Arbeitsförderung zugelassen sein.

HINWEIS

Hierüber informiert Sie der Bildungsträger oder das je- weilige Bildungsangebot in den Portalen der Aus- und Weiterbildungssuche.

1.6 Nachträglicher Erwerb des Hauptschulab-

schlusses

Auf den nachträglichen Erwerb eines Hauptschulab- schlusses oder eines gleichwertigen Schulabschlusses besteht unter bestimmten Voraussetzungen ein Rechtsanspruch. Die Förderung des Hauptschulab- schlusses erfolgt in der Regel in Kombination mit einer zuvor individuell festgelegten beruflichen Weiterbil- dungsmaßnahme.

1.7 Erwerb von Grundkompetenzen

Ab 01.07.2023 können Kunden und Kundinnen zur Vor- bereitung auf eine Weiterbildung bzw. zur Verbesse- rung der Beschäftigungsfähigkeit, Förderleistungen zum Erwerb notwendiger Grundkompetenzen in den Bereichen Lesen, Schreiben, Mathematik und Informa- tions­ und Kommunikationstechnologien erhalten.

14

1 Voraussetzungen zur Förderung

1.8 Umschulungsbegleitende Hilfen

Metadata: documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf
LEN: 1909


HINWEIS

Hierüber informiert Sie der Bildungsträger oder das je- weilige Bildungsangebot in den Portalen der Aus- und Weiterbildungssuche.

1.6 Nachträglicher Erwerb des Hauptschulab-

schlusses

Auf den nachträglichen Erwerb eines Hauptschulab- schlusses oder eines gleichwertigen Schulabschlusses besteht unter bestimmten Voraussetzungen ein Rechtsanspruch. Die Förderung des Hauptschulab- schlusses erfolgt in der Regel in Kombination mit einer zuvor individuell festgelegten beruflichen Weiterbil- dungsmaßnahme.

1.7 Erwerb von Grundkompetenzen

Ab 01.07.2023 können Kunden und Kundinnen zur Vor- bereitung auf eine Weiterbildung bzw. zur Verbesse- rung der Beschäftigungsfähigkeit, Förderleistungen zum Erwerb notwendiger Grundkompetenzen in den Bereichen Lesen, Schreiben, Mathematik und Informa- tions­ und Kommunikationstechnologien erhalten.

14

1 Voraussetzungen zur Förderung

1.8 Umschulungsbegleitende Hilfen

Umschulungsbegleitende Hilfen werden in Verbindung mit betrieblichen Einzelumschulungen gefördert. Eine betriebliche Einzelumschulung findet wie die duale Be- rufsausbildung in einem Betrieb statt. Sie ist erwachse- nengerecht um ein Drittel gegenüber einer entspre- chenden Berufsausbildung verkürzt. Umschulungsbegleitende Hilfen umfassen z. B. Nach- hilfeunterricht für Berufsschulfächer, Nachbereitung von Lernstoff.

15

2 Wie finden Sie den passenden Lehrgang?

2 Wie finden Sie den passen-

den Lehrgang?

Sie haben von Ihrer Agentur für Arbeit / Ihrem Jobcenter einen Bildungsgutschein erhalten. Darin sind das Bildungs ziel und die Qualifizierungsinhalte festgelegt, die Ihre Eingliederungschancen auf dem Arbeitsmarkt entscheidend verbessern sollen. Es liegt nun an Ihnen, den für Sie passenden Lehrgang bei einem Bildungs- träger Ihrer Wahl auszuwählen. Damit Sie die richtige Auswahl treffen können, stehen Ihnen vielfältige Infor- mationsmöglichkeiten zur Verfügung: • Wenn es um Ihre berufliche Weiterentwicklung geht,

Metadata: documents_pdf/merkblatt-6-weiterbildung_ba015381.pdf
LEN: 1957


In [92]:
import pypdf

with open("documents_pdf/ba146349.pdf", "rb") as fp:
    # Create a PDF object
    pdf = pypdf.PdfReader(fp)

In [103]:
from pypdf import PdfReader

reader = PdfReader("documents_pdf/ba146349.pdf")

number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()

In [104]:
number_of_pages

24

In [114]:
display(Markdown(reader.pages[2].extract_text()))

KiZ – Der Zuschlag zum Kindergeld
Der Kinderzuschlag ist eine Leistung für Familien mit kleinem 
Einkommen.
Den Kinderzuschlag können Eltern nur bekommen, wenn sie genug 
für sich selbst verdienen, aber das Einkommen nicht oder nur knapp für ihre gesamte Familie reicht. Wenn die Wohnkosten besonders hoch sind oder mehrere Kinder im Haushalt wohnen, kann auch bis in mittlere Einkommensbereiche hinein ein reduzierter Kinderzuschlag bezogen werden.
Der Kinderzuschlag beträgt monatlich bis zu 185  Euro je Kind. Ihr 
Einkommen und Vermögen und das Ihres Kindes werden auf den Kinderzuschlag teilweise angerechnet und reduzieren die Höhe des Kinderzuschlags.
Der Kinderzuschlag wird für 6  Monate bewilligt. Ändern sich in diesen 
6 Monaten Ihr Einkommen oder Ihre Wohnkosten, hat das keinen 
Einfluss auf den Kinderzuschlag.
3Info
Wenn Sie Kinderzuschlag bekommen, stehen Ihnen auch Bildungs- und Teilhabeleistungen - wie das kostenlose Mittagessen in KiTa und Schule oder 
das Schulbedarfspaket in Höhe von 150  Euro pro 
Schuljahr - zu. Außerdem müssen Sie keine KiTa-Gebühren zahlen.

# Nodes parser

In [210]:
text_a1 = """
Natural language processing is an interdisciplinary subfield of computer science and linguistics. It is primarily concerned with giving computers the ability to support and manipulate speech.
Natural language processing is an interdisciplinary subfield of computer science and linguistics It is primarily concerned with giving computers the ability to support and manipulate speech.
"""

In [213]:
from llama_index import Document
from llama_index.node_parser import SimpleNodeParser

node_parser = SimpleNodeParser.from_defaults(chunk_size=20, chunk_overlap=5)

nodes = node_parser.get_nodes_from_documents([Document(text=text_a1)], show_progress=False)

Metadata length (0) is close to chunk size (20). Resulting chunks are less than 50 tokens. Consider increasing the chunk size or decreasing the size of your metadata to avoid this.


In [217]:
for i in range(len(nodes)):
    display(Markdown(nodes[i].text))
    print("="*50)

Natural language processing is an interdisciplinary subfield of computer science and linguistics.

It is primarily concerned with giving computers the ability to support and manipulate speech.
Natural language processing is

Natural language processing is an interdisciplinary subfield of computer science and linguistics It is primarily concerned with

It is primarily concerned with giving computers the ability to support and manipulate speech.

Overlap only if there is not period "." in the text to overlap, otherwise it splits the sentences.

In [124]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex, ServiceContext
from llama_index.node_parser import SimpleNodeParser

docs_llama = SimpleDirectoryReader(input_files=["documents_pdf/ba146349.pdf"]).load_data()

node_parser = SimpleNodeParser.from_defaults(chunk_size=1024, chunk_overlap=20)
service_context = ServiceContext.from_defaults(node_parser=node_parser)

index = VectorStoreIndex.from_documents(docs_llama, service_context=service_context)

In [132]:
docs_llama

[Document(id_='aefc9daa-e452-4b40-92a8-a5176dcc7e19', embedding=None, metadata={'page_label': '1', 'file_name': 'ba146349.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='592bb4a567ff530dbc0607be761d79990988384fdd21bdcda882ef64fc386f94', text=' Familienkasse\nWir helfen Familien.\nMerkblatt\nKinderzuschlag\nDer Zuschlag zum Kindergeld für Familien  \nmit kleinem Einkommen', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='602c9991-d6d6-49ec-bf7f-eab20125d6ec', embedding=None, metadata={'page_label': '2', 'file_name': 'ba146349.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='a5391d9b6b68128c685a173de8d7e5231e52fcd67e4d1ca305065065f6bf2140', text='Inhaltsverzeichnis  \n Seite \nKiZ – der Zuschlag zum Kindergeld  3\nZu diesem Merkblatt  4\n1. Wer hat einen Anspruch auf Kinderzuschlag?  

In [133]:
from llama_index.node_parser import SimpleNodeParser


# parse nodes
parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(docs_llama)

In [139]:
docs_llama[0].get_text()

' Familienkasse\nWir helfen Familien.\nMerkblatt\nKinderzuschlag\nDer Zuschlag zum Kindergeld für Familien  \nmit kleinem Einkommen'

In [140]:
nodes[0].get_content()

'Familienkasse\nWir helfen Familien.\nMerkblatt\nKinderzuschlag\nDer Zuschlag zum Kindergeld für Familien  \nmit kleinem Einkommen'